# Leveraging Gen AI for SAT Prep

I’ll start with sharing my motivation for creating this notebook. When I was preparing for SAT mid-2023, I struggled to find extra study material for the new Digital SAT (launched in Spring of 2023), especially for the reading and writing section, which is tougher to crack compared to the math section. I had an idea: why not ask ChatGPT? But to my surprise, the SAT-style questions it generated were lackluster and I knew that I could be the only one, leading me to dive deeper into researching how I can leverage Gen AI for SAT prep.

I ended up figuring out an approach to leverage Llama 3 Instruct models for building my proficiency in word in context type of questions. Through that I was able to build my vocabulary, which is the critical aspect in scoring well in the reading and writing section. I scored 760 out of 800 in my first SAT attempt, which is 99th percentile among all test takers.

You don't need a GPU device to run this notebook. Just CPU based Colab instance is sufficient.

We'll start with installing transformers and torch libraries

In [ ]:
!pip install transformers
!pip install torch

In [72]:
import transformers
import torch
print(transformers.__version__)
print(torch.__version__)

4.47.1
2.5.1+cu121


To run this notebook, you will need a Hugging Face (HF) token as Llama models are gated and require users to accept Meta’s usage terms. To get a token, you will have to provide your contact information in the HF model page (https://huggingface.co/meta-llama/Llama-3.1-70B-Instruct), accept the terms, and you will receive an email once the access has been approved. Followed by that, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab by navigating to the "Secrets" tab in the left panel and creating a new secret named "HF_TOKEN" and paste your Hugging Face token as the value. Restart you session thereafer.

We use Llama 3.1-8B model due to resource restrictions. If you have a powerful machine, you can leverage the 70B model.

In [54]:
#model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
model_id = "meta-llama/Llama-3.1-8B"

### SAT Vocabulary Dataset

I created a SAT vocabulary dataset containing 500+ words. You can download the csv from my Git Repository. I used a CSV fromat that can be converted to Hugging Face Datasets if need be.

In [79]:
import pandas as pd
import random
from random import randrange
vocab_df = pd.read_csv('sat_vocabulary.csv')
print("Sample word: {} ".format(vocab['word'][0]))

Sample word: abrupt 


### SAT style pragraph Genre

I created a Genre dataset that can be used to instruct the model to create context based on a given Genre.

In [74]:
genre_df = pd.read_csv('sat_genre.csv')
print("Sample genre: {} ".format(genre['genre'][0]))

Sample genre: poetry 


The following step will download the model weights and will take about 8 - 10 mins.

In [55]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cpu


Function Genereate SAT style paragraph

In [95]:
paragraph_length=250
#function for generating a paragraph given a SAT word and Genre
def generate_paragraph(word, genre, debug):
  if debug: print("word: {}, genre: {}".format(word, genre))
  instruction = "generate a {} word paragraph on {} with the word {} in it".format(paragraph_length, genre, word)
  if debug: print("instruction: {}".format(instruction))
  output = pipeline(
    instruction,
    max_new_tokens=paragraph_length,
  )
  return output[0]

#function for maskign the SAT word from the generated paragraph
def mask_word(word, paragraph):
  return paragraph.replace(word, '_____')

#function to create a list of answer choices that has the correct work and three incorrect word
def get_answer_choices(word):
  ans_choices = [word]
  for i in range(20):
    temp_ans = vocab_df['word'][randrange(vocab_df.shape[0])]
    if temp_ans not in ans_choices:
      ans_choices.append(temp_ans)
    if(len(ans_choices) >= 4):
      break
  random.shuffle(ans_choices)
  return ans_choices

#this function calls all other functions
def run_prep (word, genre, debug=False):
  para = generate_paragraph(word, genre, debug)
  para = mask_word(word, para)
  return [para, get_answer_choices(word)]

In [94]:
sat_word = vocab_df['word'][randrange(vocab_df.shape[0])]
sat_genre = genre_df['genre'][randrange(genre_df.shape[0])]
print(run_prep(sat_word, sat_genre, True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


word: defends, genre: poetry
instruction: generate a 250 word paragraph on poetry with the word defends in it


KeyboardInterrupt: 